In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import numpy as np

## 2.

### Prepare data for each gate

In [4]:
# Define the truth tables for the gates
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and = np.array([0, 0, 0, 1])
y_or = np.array([0, 1, 1, 1])
y_xor = np.array([0, 1, 1, 0])
y_nand = np.array([1, 1, 1, 0])

### Create models

In [11]:
# Initialize models
logistic_regression = LogisticRegression()
svc = SVC() # rbf as kernel default

### Train the models 

In [12]:
# Train and evaluate models for each gate
results = {}
for gate_name, y in zip(['AND', 'OR', 'XOR', 'NAND'], [y_and, y_or, y_xor, y_nand]):
    # Train Logistic Regression
    logistic_regression.fit(X, y)
    y_pred_lr = logistic_regression.predict(X)
    accuracy_lr = accuracy_score(y, y_pred_lr)

    # Train SVC
    svc.fit(X, y)
    y_pred_svc = svc.predict(X)
    accuracy_svc = accuracy_score(y, y_pred_svc)

    # Store results
    results[gate_name] = {
        'Logistic Regression': {'predictions': y_pred_lr, 'accuracy': accuracy_lr},
        'SVC': {'predictions': y_pred_svc, 'accuracy': accuracy_svc}
    }


In [13]:
print(results)

{'AND': {'Logistic Regression': {'predictions': array([0, 0, 0, 0]), 'accuracy': 0.75}, 'SVC': {'predictions': array([0, 0, 0, 1]), 'accuracy': 1.0}}, 'OR': {'Logistic Regression': {'predictions': array([1, 1, 1, 1]), 'accuracy': 0.75}, 'SVC': {'predictions': array([0, 1, 1, 1]), 'accuracy': 1.0}}, 'XOR': {'Logistic Regression': {'predictions': array([0, 0, 0, 0]), 'accuracy': 0.5}, 'SVC': {'predictions': array([0, 1, 1, 0]), 'accuracy': 1.0}}, 'NAND': {'Logistic Regression': {'predictions': array([1, 1, 1, 1]), 'accuracy': 0.75}, 'SVC': {'predictions': array([1, 1, 1, 0]), 'accuracy': 1.0}}}


* Logistic Regression: This model struggles with these logic gates because it is a linear model and tends to predict the majority class, especially in imbalanced datasets. It cannot capture the non-linear relationships required for gates like XOR. 
* SVC with RBF Kernel: This model performs perfectly for all gates because the RBF kernel can capture non-linear patterns, which is crucial for learning the behavior of gates like XOR and NAND

## 3.

### Create 2000 random imbalanced data